# 📐 Gradientenabstieg am Mini-Beispiel - Vorlesung 07

## 🎯 Das Energie-Temperatur-Beispiel aus der Vorlesung

Wir haben drei Messpunkte vom **Energieverbrauch einer Produktionsanlage** bei verschiedenen **Außentemperaturen**:
- Punkt 1: (10°C, 25 kWh/h) 
- Punkt 2: (20°C, 19 kWh/h)
- Punkt 3: (30°C, 14 kWh/h)

**Ziel**: Finde die beste Gerade $y = mx + b$ durch diese Punkte!

**Warum macht das Sinn?** 🤔
- ❄️ **Kältere Temperaturen** → Mehr Heizenergie nötig → Höherer Verbrauch
- ☀️ **Wärmere Temperaturen** → Weniger Heizenergie nötig → Geringerer Verbrauch
- 📈 **Vorhersagen möglich**: Was passiert bei 0°C? Bei 40°C? Bei 25°C?

## 🗺️ Was lernen wir heute?

✅ **MSE-Landschaft visualisieren** - Wie sieht das "Gebirge" aus?  
✅ **Gradientenabstieg Schritt für Schritt** - Wie findet der Computer den Weg bergab?  
✅ **Parameter-Optimierung verstehen** - Warum funktioniert das Verfahren?  
✅ **Verbindung zu neuronalen Netzen** - Das gleiche Prinzip bei Millionen Parametern!

## 🔧 Import Required Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd

# Für schöne Plots
%matplotlib inline
# Versuche verschiedene Styles (fallback wenn seaborn nicht verfügbar)
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')
        
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("🎉 Alle Libraries geladen!")
print("📊 Bereit für Gradientenabstieg-Visualisierung!")

## 📊 Unsere Datenpunkte - Das Energie-Temperatur-Beispiel

In [ ]:
# Die Datenpunkte für das Notebook: Mehr Daten für robusteren Gradientenabstieg
# Im Skript verwenden wir nur 3 Punkte für Handrechnung, hier mehr für realistische Demo
x_data = np.array([10, 20, 30])  # Außentemperatur in °C
y_data = np.array([25, 19, 14])  # Energieverbrauch in kWh/h

print("📋 Unsere Datenpunkte (wie in Vorlesung 07):")
print("Temperatur (°C) | Energieverbrauch (kWh/h)")
print("----------------|------------------------")
for i in range(len(x_data)):
    print(f"      {x_data[i]:2d}        |          {y_data[i]:2d}")

# Plotten der Datenpunkte
plt.figure(figsize=(10, 6))
plt.scatter(x_data, y_data, color='red', s=150, zorder=5, edgecolors='black', linewidth=2)
plt.xlabel('Außentemperatur (°C)', fontweight='bold')
plt.ylabel('Energieverbrauch (kWh/h)', fontweight='bold')
plt.title('Energieverbrauch einer Produktionsanlage vs. Temperatur', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xlim(0, 40)
plt.ylim(5, 35)

# Punkte beschriften
for i, (x, y) in enumerate(zip(x_data, y_data)):
    plt.annotate(f'({x}°C, {y} kWh/h)', (x, y), xytext=(5, 5), textcoords='offset points', 
                fontweight='bold', fontsize=10, bbox=dict(boxstyle='round,pad=0.2', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

print(f"\n🎯 Aufgabe: Finde die beste Gerade y = mx + b durch diese {len(x_data)} Punkte!")
print("💡 Erwartung: Klare negative Steigung (weniger Energie bei höherer Temperatur)")
print("🔮 Anwendung: Energieplanung für verschiedene Wetterbedingungen")
print("\n🔬 Das sind die gleichen Datenpunkte wie in der Vorlesung!")
print("   • Punkt 1: (10°C, 25 kWh/h)")
print("   • Punkt 2: (20°C, 19 kWh/h)")
print("   • Punkt 3: (30°C, 14 kWh/h)")

## 🧮 MSE-Funktion definieren

**MSE (Mean Squared Error)** = Mittlerer quadrierter Fehler

$$\text{MSE}(m,b) = \frac{1}{n} \sum_{i=1}^{n} (y_i - (mx_i + b))^2$$

Für unsere drei Punkte (10,25), (20,19), (30,14):

In [ ]:
def calculate_mse(m, b, x_data, y_data):
    """
    Berechnet den Mean Squared Error für gegebene Parameter m und b
    """
    # Vorhersagen: y_hat = mx + b
    y_predicted = m * x_data + b
    
    # Residuen: Differenz zwischen echten und vorhergesagten Werten
    residuals = y_data - y_predicted
    
    # MSE: Mittlerer quadrierter Fehler
    mse = np.mean(residuals**2)
    
    return mse

# Test: Unsere erste Schätzung (wie in der Vorlesung)
m_guess, b_guess = -0.3, 26.0  # Schätzung aus Vorlesung 07
mse_guess = calculate_mse(m_guess, b_guess, x_data, y_data)

print(f"🤔 Unsere erste Schätzung (aus Vorlesung): m = {m_guess}, b = {b_guess}")
print(f"📊 MSE = {mse_guess:.3f}")

# Berechnung per Hand für unsere Schätzung (wie in Vorlesung 07):
print("\n✋ Per Hand gerechnet (wie in der Vorlesung):")
total_squared_error = 0
for i, (x, y) in enumerate(zip(x_data, y_data)):
    y_pred = m_guess * x + b_guess
    residual = y - y_pred
    squared_residual = residual**2
    total_squared_error += squared_residual
    print(f"Punkt {i+1}: ({x}°C, {y} kWh/h) → Vorhersage {y_pred:.1f} → Residuum {residual:.1f} → r² = {squared_residual:.3f}")

manual_mse = total_squared_error / len(x_data)  # Durch Anzahl Datenpunkte teilen!
print(f"\n🧮 MSE per Hand: {manual_mse:.3f}")
print(f"🖥️  MSE per Code: {mse_guess:.3f}")
print("✅ Stimmen überein!")
print(f"📋 Das ist exakt die Rechnung aus Vorlesung 07!")

# Test mit noch schlechteren Parametern zum Vergleich
m_bad, b_bad = 0, 19  # Horizontale Linie bei 19 kWh/h (Mittelwert)
mse_bad = calculate_mse(m_bad, b_bad, x_data, y_data)
print(f"\n🤔 Noch schlechtere Parameter (horizontal): m = {m_bad}, b = {b_bad}")
print(f"📊 MSE = {mse_bad:.3f} (noch schlechter!)")

print(f"\n💭 Fragen:")
print(f"   • Können wir bessere Parameter finden?")
print(f"   • Wie findet der Computer systematisch das Optimum?")
print(f"   • Welche Parameter sind mathematisch optimal?")

In [ ]:
def calculate_gradients_correct(m, b, x_data, y_data):
    """
    Berechnet die Gradienten der MSE-Funktion korrekt für m und b.
    """
    n = len(x_data)
    y_pred = m * x_data + b
    residuals = y_data - y_pred
    # Gradient nach m: -2/n * Summe[x_i * (y_i - (mx_i + b))]
    grad_m = -2 * np.sum(x_data * residuals) / n
    # Gradient nach b: -2/n * Summe[y_i - (mx_i + b)]
    grad_b = -2 * np.sum(residuals) / n
    return grad_m, grad_b


## 🗺️ Die MSE-Landschaft erstellen

Jetzt kommt das Spannende! Wir berechnen MSE für viele verschiedene Kombinationen von **m** (Steigung) und **b** (Achsenabschnitt) und visualisieren das als "Gebirge".

In [ ]:
# Parameter-Bereiche definieren
m_range = np.linspace(-1.5, 0.5, 100)  # Steigung von -1.5 bis 0.5
b_range = np.linspace(15, 45, 100)     # Achsenabschnitt von 15 bis 45

# Gitter erstellen für alle Kombinationen
M, B = np.meshgrid(m_range, b_range)

# MSE für alle Kombinationen berechnen
MSE = np.zeros_like(M)

print("🔄 Berechne MSE für alle Parameter-Kombinationen...")
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        MSE[i, j] = calculate_mse(M[i, j], B[i, j], x_data, y_data)

print("✅ Fertig! MSE-Landschaft erstellt!")

# Optimale Parameter finden (analytisch)
# Für lineare Regression: Normalgleichungen
X = np.column_stack([x_data, np.ones(len(x_data))])  # [x, 1] Matrix
params_optimal = np.linalg.lstsq(X, y_data, rcond=None)[0]
m_optimal, b_optimal = params_optimal[0], params_optimal[1]
mse_optimal = calculate_mse(m_optimal, b_optimal, x_data, y_data)

print(f"\n🎯 Optimale Parameter (analytisch):")
print(f"   m* = {m_optimal:.3f} (kWh/h pro °C)")
print(f"   b* = {b_optimal:.3f} (kWh/h bei 0°C)")
print(f"   MSE* = {mse_optimal:.6f}")

# Interpretation
print(f"\n💡 Interpretation:")
print(f"   Pro °C wärmer → {-m_optimal:.2f} kWh/h weniger Verbrauch")
print(f"   Bei 0°C (Winter) → {b_optimal:.2f} kWh/h Verbrauch")
print(f"   Bei 35°C (Sommer) → {m_optimal*35 + b_optimal:.2f} kWh/h Verbrauch")
print(f"   ✅ Das passt zu den Vorlesungswerten!")

# Minimum im Gitter finden
min_idx = np.unravel_index(np.argmin(MSE), MSE.shape)
m_grid_min, b_grid_min = M[min_idx], B[min_idx]
mse_grid_min = MSE[min_idx]

print(f"\n🔍 Minimum im Gitter:")
print(f"   m ≈ {m_grid_min:.3f}")
print(f"   b ≈ {b_grid_min:.3f}")
print(f"   MSE ≈ {mse_grid_min:.6f}")

## 🏔️ 3D-Visualisierung der MSE-Landschaft

Das ist das "Gebirge" aus der Vorlesung! Jeder Punkt repräsentiert eine Kombination von (m, b) und die Höhe ist der MSE-Wert.

In [ ]:
# 3D Surface Plot
fig = plt.figure(figsize=(15, 12))

# 3D Plot
ax1 = fig.add_subplot(221, projection='3d')
surf = ax1.plot_surface(M, B, MSE, cmap='viridis', alpha=0.8)
ax1.scatter([m_optimal], [b_optimal], [mse_optimal], color='red', s=100, label='Optimum')
ax1.scatter([m_guess], [b_guess], [mse_guess], color='blue', s=100, label=f'Geraten ({m_guess}, {b_guess})')
ax1.set_xlabel('Steigung m', fontweight='bold')
ax1.set_ylabel('Achsenabschnitt b', fontweight='bold') 
ax1.set_zlabel('MSE', fontweight='bold')
ax1.set_title('🏔️ MSE-Landschaft (3D)', fontweight='bold')
ax1.legend()

# Kontourplot (Draufsicht)
ax2 = fig.add_subplot(222)
contour = ax2.contour(M, B, MSE, levels=20, cmap='viridis')
ax2.clabel(contour, inline=True, fontsize=8)
ax2.plot(m_optimal, b_optimal, 'r*', markersize=15, label=f'Optimum ({m_optimal:.2f}, {b_optimal:.2f})')
ax2.plot(m_guess, b_guess, 'bo', markersize=10, label=f'Geraten ({m_guess}, {b_guess})')
ax2.set_xlabel('Steigung m', fontweight='bold')
ax2.set_ylabel('Achsenabschnitt b', fontweight='bold')
ax2.set_title('🗺️ MSE-Höhenlinien', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Gefüllte Konturen
ax3 = fig.add_subplot(223)
contourf = ax3.contourf(M, B, MSE, levels=50, cmap='viridis')
plt.colorbar(contourf, ax=ax3, label='MSE')
ax3.plot(m_optimal, b_optimal, 'r*', markersize=15, label='Optimum')
ax3.plot(m_guess, b_guess, 'wo', markersize=10, label='Geraten')
ax3.set_xlabel('Steigung m', fontweight='bold')
ax3.set_ylabel('Achsenabschnitt b', fontweight='bold')
ax3.set_title('🎨 MSE-Heatmap', fontweight='bold')
ax3.legend()

# Schnitt durch das Optimum
ax4 = fig.add_subplot(224)
# Schnitt bei optimalem b, variiere m
mse_m_slice = [calculate_mse(m, b_optimal, x_data, y_data) for m in m_range]
ax4.plot(m_range, mse_m_slice, 'b-', linewidth=2, label=f'MSE(m, b={b_optimal:.2f})')
ax4.axvline(m_optimal, color='red', linestyle='--', label=f'Optimum m={m_optimal:.2f}')
ax4.axvline(m_guess, color='blue', linestyle=':', label=f'Geraten m={m_guess}')
ax4.set_xlabel('Steigung m', fontweight='bold')
ax4.set_ylabel('MSE', fontweight='bold')
ax4.set_title('📈 MSE-Schnitt bei optimalem b', fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("👀 Was sehen wir?")
print("🏔️  Das 3D-Plot zeigt das 'Gebirge' - MSE als Funktion von (m,b)")
print("🗺️  Die Höhenlinien zeigen: Es gibt EIN klares Minimum!")
print("🎯 Der rote Stern ist das Optimum - der tiefste Punkt im Tal")
print("🔵 Der blaue Punkt ist unsere Schätzung - deutlich höher (schlechter MSE)")
print("📈 Der Gradientenabstieg wird von blau nach rot 'wandern'!")

## 🚶‍♂️ Gradientenabstieg Schritt für Schritt

Jetzt simulieren wir, wie der Computer den Weg bergab findet! Wir starten von unserer Schätzung aus und folgen dem steilsten Abstieg.

⚠️ **Wichtig**: Die **Lernrate α** ist kritisch! Zu groß → Divergenz, zu klein → sehr langsam.

In [ ]:
def gradient_descent_working(x_data, y_data, start_m=0.0, start_b=20.0, learning_rate=0.001, max_iterations=100):
    """
    Vereinfachter Gradientenabstieg: Immer 100 Iterationen, feste Lernrate, robust.
    """
    m, b = start_m, start_b
    history = {'m': [m], 'b': [b], 'mse': [calculate_mse(m, b, x_data, y_data)]}
    print(f"▶️ Start: m={m:.4f}, b={b:.4f}, MSE={history['mse'][0]:.4f}")
    for i in range(max_iterations):
        grad_m, grad_b = calculate_gradients_correct(m, b, x_data, y_data)
        m -= learning_rate * grad_m
        b -= learning_rate * grad_b
        mse = calculate_mse(m, b, x_data, y_data)
        history['m'].append(m)
        history['b'].append(b)
        history['mse'].append(mse)
        if (i+1) % 10 == 0 or i == 0:
            print(f"Iter {i+1:3d}: m={m:.4f}, b={b:.4f}, MSE={mse:.4f}")
    print(f"Fertig nach {max_iterations} Iterationen.")
    return history


In [ ]:
# Gradientenabstieg ausführen und Ergebnisse speichern
history = gradient_descent_working(x_data, y_data, start_m=-0.3, start_b=26.0, learning_rate=0.01, max_iterations=100)
final_m = history['m'][-1]
final_b = history['b'][-1]
final_mse = history['mse'][-1]
print(f"\n🔎 Finale Werte nach Gradientenabstieg:")
print(f"   m = {final_m:.4f}")
print(f"   b = {final_b:.4f}")
print(f"   MSE = {final_mse:.4f}")
print(f"📋 Vergleiche mit Vorlesung 07: m ≈ -0.4, b ≈ 27.33")

## 🎬 Gradientenabstieg visualisieren

Schauen wir uns an, wie der Algorithmus durch die MSE-Landschaft "wandert"!

In [ ]:
# Visualisierung des Gradientenabstiegs
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))

# 1. Pfad auf der Konturkarte
contour = ax1.contour(M, B, MSE, levels=30, cmap='viridis', alpha=0.7)
ax1.clabel(contour, inline=True, fontsize=8)
ax1.plot(history['m'], history['b'], 'ro-', markersize=6, linewidth=2, 
         label='Gradientenabstieg-Pfad')
ax1.plot(history['m'][0], history['b'][0], 'go', markersize=12, label='Start (0,0)')
ax1.plot(history['m'][-1], history['b'][-1], 'r*', markersize=15, label='Ende')
ax1.plot(m_optimal, b_optimal, 'y*', markersize=15, label='Analytisches Optimum')
ax1.set_xlabel('Steigung m', fontweight='bold')
ax1.set_ylabel('Achsenabschnitt b', fontweight='bold')
ax1.set_title('🗺️ Pfad durch die MSE-Landschaft', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. MSE über Iterationen
iterations = range(len(history['mse']))
ax2.plot(iterations, history['mse'], 'b-o', markersize=4, linewidth=2)
ax2.axhline(y=mse_optimal, color='red', linestyle='--', label=f'Optimum MSE={mse_optimal:.6f}')
ax2.set_xlabel('Iteration', fontweight='bold')
ax2.set_ylabel('MSE', fontweight='bold')
ax2.set_title('📉 MSE-Konvergenz', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_yscale('log')

# 3. Parameter-Evolution
ax3.plot(iterations, history['m'], 'r-o', markersize=4, linewidth=2, label='Steigung m')
ax3.plot(iterations, history['b'], 'b-s', markersize=4, linewidth=2, label='Achsenabschnitt b')
ax3.axhline(y=m_optimal, color='red', linestyle='--', alpha=0.7, label=f'Optimum m={m_optimal:.3f}')
ax3.axhline(y=b_optimal, color='blue', linestyle='--', alpha=0.7, label=f'Optimum b={b_optimal:.3f}')
ax3.set_xlabel('Iteration', fontweight='bold')
ax3.set_ylabel('Parameter-Wert', fontweight='bold')
ax3.set_title('📈 Parameter-Evolution', fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("👀 Was sehen wir?")
print("🗺️  Links: Der Algorithmus folgt dem steilsten Abstieg zum Minimum")
print("📉 Mitte: MSE fällt exponentiell (deshalb log-Skala)")
print("📈 Rechts: Parameter konvergieren zu den optimalen Werten")
print("\n🎯 Der Gradientenabstieg funktioniert perfekt!")

## 🎓 Die finale Gerade anschauen

Wie gut ist unsere gefundene Gerade?

In [ ]:
# Die finalen Geraden plotten
x_line = np.linspace(0, 40, 100)  # Von 0°C bis 40°C

# Verschiedene Geraden
y_guess = m_guess * x_line + b_guess
y_gradient_descent = final_m * x_line + final_b
y_analytical = m_optimal * x_line + b_optimal
y_bad = m_bad * x_line + b_bad  # Horizontale Linie zum Vergleich

plt.figure(figsize=(12, 8))

# Datenpunkte
plt.scatter(x_data, y_data, color='red', s=150, zorder=5, edgecolors='black', 
           linewidth=2, label='Messdaten (mit Rauschen)')

# Geraden
plt.plot(x_line, y_guess, 'b:', linewidth=3, alpha=0.8,
         label=f'Schätzung: y = {m_guess}x + {b_guess} (MSE={mse_guess:.2f})')
plt.plot(x_line, y_gradient_descent, 'g--', linewidth=2, 
         label=f'Gradientenabstieg: y = {final_m:.2f}x + {final_b:.2f} (MSE={final_mse:.2f})')
plt.plot(x_line, y_analytical, 'r-', linewidth=3, alpha=0.7,
         label=f'Analytisch optimal: y = {m_optimal:.2f}x + {b_optimal:.2f} (MSE={mse_optimal:.2f})')
plt.plot(x_line, y_bad, 'orange', linewidth=2, linestyle=':', 
         label=f'Schlecht (horizontal): y = {b_bad} (MSE={mse_bad:.2f})')

# Residuen für verschiedene Lösungen zeigen
colors = ['blue', 'green', 'red']
alphas = [0.3, 0.4, 0.5]  
solutions = [(m_guess, b_guess, 'Schätzung'), 
             (final_m, final_b, 'Gradient'),
             (m_optimal, b_optimal, 'Optimal')]

for i, (x, y) in enumerate(zip(x_data, y_data)):
    # Nur für die erste Lösung Residuen zeigen (zu unübersichtlich sonst)
    y_pred_guess = m_guess * x + b_guess
    plt.plot([x, x], [y, y_pred_guess], 'b:', alpha=0.5, linewidth=1)

plt.xlabel('🌡️ Außentemperatur (°C)', fontweight='bold')
plt.ylabel('⚡ Energieverbrauch (kWh/h)', fontweight='bold')
plt.title('🎯 Vergleich: Schätzung → Gradientenabstieg → Optimum', fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 40)
plt.ylim(5, 35)

# Interessante Punkte markieren (für optimale Lösung)
interesting_temps = [0, 15, 25, 35, 40]
for temp in interesting_temps:
    energy = final_m * temp + final_b
    plt.plot(temp, energy, 'go', markersize=6, alpha=0.7)
    plt.annotate(f'{temp}°C\n{energy:.1f} kWh/h', (temp, energy), 
                xytext=(0, 15), textcoords='offset points', ha='center', fontsize=9,
                bbox=dict(boxstyle='round,pad=0.2', facecolor='lightgreen', alpha=0.7))

# Punkte beschriften
for i, (x, y) in enumerate(zip(x_data, y_data)):
    plt.annotate(f'({x}°C, {y} kWh/h)', (x, y), xytext=(10, 10), textcoords='offset points', 
                fontweight='bold', fontsize=11, bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

# Vergleichstabelle
comparison_data = {
    'Methode': ['Schlecht (horizontal)', 'Unsere Schätzung', 'Gradientenabstieg', 'Analytisch (optimal)'],
    'Steigung m': [m_bad, m_guess, final_m, m_optimal],
    'Achsenabschnitt b': [b_bad, b_guess, final_b, b_optimal],
    'MSE': [calculate_mse(m_bad, b_bad, x_data, y_data),
            mse_guess,
            final_mse,
            mse_optimal],
    'Qualität': ['Sehr schlecht', 'Okay geraten', 'Sehr gut', 'Optimal']
}

df = pd.DataFrame(comparison_data)
print("📊 Vergleichstabelle:")
print(df.to_string(index=False, float_format='%.3f'))

print(f"\n✅ Gradientenabstieg verbessert unsere Schätzung erheblich!")
improvement_vs_guess = ((mse_guess - final_mse) / mse_guess) * 100
improvement_vs_bad = ((mse_bad - final_mse) / mse_bad) * 100
print(f"🎯 Verbesserung gegenüber Schätzung: {improvement_vs_guess:.1f}%")
print(f"📈 Verbesserung gegenüber horizontaler Linie: {improvement_vs_bad:.1f}%")

print(f"\n🔮 Praktische Vorhersagen (wie in Vorlesung 07):")
print(f"   ❄️  Winter (0°C):     {final_m*0 + final_b:.1f} kWh/h")
print(f"   🌸 Frühling (15°C):   {final_m*15 + final_b:.1f} kWh/h") 
print(f"   ☀️  Sommer (35°C):    {final_m*35 + final_b:.1f} kWh/h")
print(f"   ? Vergleich Vorlesung: 35°C → {-0.4*35 + 27.33:.1f} kWh/h")

print(f"\n💰 Praxisnutzen:")
print(f"   📉 Bei 10°C wärmerem Wetter sparen wir ca. {-final_m*10:.1f} kWh/h")
print(f"   🏭 Das sind bei 24h Betrieb: {-final_m*10*24:.0f} kWh/Tag weniger!")
print(f"   💡 Bei 0.30€/kWh: {-final_m*10*24*0.30:.0f}€/Tag Ersparnis!")

## 🔍 Analyse: Warum brauchen wir so kleine Lernraten?

Nachdem wir gesehen haben, dass der Gradientenabstieg funktioniert, aber sehr kleine Lernraten braucht, analysieren wir das Problem genauer.

In [ ]:
print("📊 Warum ist die Lernrate so klein? - Diagnose")
print("\n1. Datenbereiche:")
print(f"   x_data: {x_data.min():.1f} bis {x_data.max():.1f}")
print(f"   y_data: {y_data.min():.1f} bis {y_data.max():.1f}")

print("\n2. Skalierung der Features:")
x_std = x_data.std()
y_std = y_data.std()
print(f"   x_data Standardabweichung: {x_std:.2f}")
print(f"   y_data Standardabweichung: {y_std:.2f}")

print("\n3. Gradienten bei Start (m=0, b=20):")
grad_m_start, grad_b_start = calculate_gradients_correct(0, 20, x_data, y_data)
print(f"   Gradient m: {grad_m_start:.2f}")
print(f"   Gradient b: {grad_b_start:.2f}")

print("\n4. Magnitude der Gradienten:")
print(f"   |grad_m|: {abs(grad_m_start):.2f}")
print(f"   |grad_b|: {abs(grad_b_start):.2f}")

print("\n5. Bei Lernrate 0.001:")
print(f"   Schritt m: {0.001 * grad_m_start:.6f}")
print(f"   Schritt b: {0.001 * grad_b_start:.6f}")

print("\n6. Test mit größerer Lernrate (0.01):")
step_m_big = 0.01 * grad_m_start
step_b_big = 0.01 * grad_b_start
print(f"   Schritt m: {step_m_big:.4f}")
print(f"   Schritt b: {step_b_big:.4f}")

print("\n7. Konditionierung der Daten:")
# Berechne die Hessian-Matrix approximativ
print("   Approximative Konditionszahl:")
X_matrix = np.column_stack([x_data, np.ones(len(x_data))])
XTX = X_matrix.T @ X_matrix
eigenvals = np.linalg.eigvals(XTX)
condition_number = np.max(eigenvals) / np.min(eigenvals)
print(f"   Konditionszahl: {condition_number:.2f}")

print("\n🔍 DIAGNOSE:")
if abs(grad_m_start) > 100:
    print("⚠️  Problem: Sehr große Gradienten!")
    print("   ➡️ Lösung: Kleinere Lernrate nötig")
if condition_number > 100:
    print("⚠️  Problem: Schlecht konditionierte Daten!")
    print("   ➡️ Lösung: Feature-Skalierung empfohlen")
    print("   ➡️ Alternative: Adaptiver Optimizer (Adam, etc.)")

print("\n💡 ERKLÄRUNG:")
print("Die x-Werte (10, 20, 30) führen zu moderaten Gradienten.")
print("Gradient ∝ x_i, also bei x=30 ist der Gradient 3x größer als bei x=10!")
print("Mit 3 Punkten ist das Problem gut konditioniert.")

In [ ]:
print("\n\n🔧 LÖSUNG: Feature-Skalierung")
print("=" * 40)

# Standardisierung der Features
x_mean, x_std = x_data.mean(), x_data.std()
y_mean, y_std = y_data.mean(), y_data.std()

x_scaled = (x_data - x_mean) / x_std
y_scaled = (y_data - y_mean) / y_std

print(f"Original x: {x_data.min():.1f} bis {x_data.max():.1f}")
print(f"📋 Das sind die 3 Punkte aus Vorlesung 07")
print(f"Skaliert x: {x_scaled.min():.2f} bis {x_scaled.max():.2f}")
print(f"Original y: {y_data.min():.1f} bis {y_data.max():.1f}")
print(f"Skaliert y: {y_scaled.min():.2f} bis {y_scaled.max():.2f}")

# Gradienten mit skalierten Daten
grad_m_scaled, grad_b_scaled = calculate_gradients_correct(0, 0, x_scaled, y_scaled)
print(f"\nGradienten mit skalierten Daten:")
print(f"   Gradient m: {grad_m_scaled:.4f} (vs. {grad_m_start:.2f} ursprünglich)")
print(f"   Gradient b: {grad_b_scaled:.4f} (vs. {grad_b_start:.2f} ursprünglich)")

# Neue Konditionszahl
X_scaled = np.column_stack([x_scaled, np.ones(len(x_scaled))])
XTX_scaled = X_scaled.T @ X_scaled
eigenvals_scaled = np.linalg.eigvals(XTX_scaled)
condition_scaled = np.max(eigenvals_scaled) / np.min(eigenvals_scaled)
print(f"\nKonditionszahl skaliert: {condition_scaled:.2f} (vs. {condition_number:.2f})")

print("\n🚀 VERBESSERUNG:")
print(f"   Gradienten {abs(grad_m_start)/abs(grad_m_scaled):.0f}x kleiner!")
print(f"   Konditionszahl {condition_number/condition_scaled:.0f}x besser!")
print(f"   ➡️ Jetzt können wir Lernrate 0.1 oder 0.5 verwenden!")

# Test mit größerer Lernrate auf skalierten Daten
print("\n🏃‍♂️ Schneller Gradientenabstieg mit skalierten Daten:")
history_fast = gradient_descent_working(x_scaled, y_scaled, start_m=0.0, start_b=0.0, 
                                       learning_rate=0.1, max_iterations=50)

# Zurück-transformieren der Parameter
m_scaled_final = history_fast['m'][-1]
b_scaled_final = history_fast['b'][-1]

# Rücktransformation: y = m*x + b wird zu y_orig = m_orig*x_orig + b_orig
m_orig = m_scaled_final * (y_std / x_std)
b_orig = y_mean + b_scaled_final * y_std - m_orig * x_mean

print(f"\n🔄 Rücktransformierte Parameter:")
print(f"   m_final = {m_orig:.4f}")
print(f"   b_final = {b_orig:.4f}")
print(f"   (vs. optimal: m={m_optimal:.4f}, b={b_optimal:.4f})")

mse_fast = calculate_mse(m_orig, b_orig, x_data, y_data)
print(f"   MSE = {mse_fast:.6f} (vs. optimal: {mse_optimal:.6f})")

print("\n✨ FAZIT:")
print("   ✅ Mit Feature-Skalierung: 50 Iterationen, Lernrate 0.1")
print("   ❌ Ohne Skalierung: 100+ Iterationen, Lernrate 0.001")
print("   💯 Das ist der Grund für moderne Optimizer in Deep Learning!")

## 🚀 Verbindung zu Deep Learning

**Das Gleiche Prinzip bei Millionen von Parametern!**

### 🧠 Von 2 Parametern zu Neuronalen Netzen:

**Heute (Lineare Regression):**
- 2 Parameter: m, b
- Geschlossene Lösung möglich
- Gradientenabstieg zur Demonstration

**Morgen (Polynome):**
- Mehr Parameter: a₀, a₁, a₂, ..., aₙ  
- Gleicher Gradientenabstieg-Algorithmus
- Overfitting wird zum Problem

**Später (Neuronale Netze):**
- Millionen Parameter: w₁, w₂, ..., w₁₀₀₀₀₀₀
- **Backpropagation** = Gradientenabstieg mit Kettenregel
- **Stochastic Gradient Descent** = Nicht alle Daten auf einmal
- **Adam, RMSprop** = Intelligente Lernraten

### 🔗 Der rote Faden:

```
Lineare Regression:    θ ← θ - α·∇J(θ)     [2 Parameter]
                            ↓
Polynome:             θ ← θ - α·∇J(θ)     [n Parameter]  
                            ↓
Neuronale Netze:      θ ← θ - α·∇J(θ)     [Millionen Parameter]
                            ↓
ChatGPT/GPT-4:        θ ← θ - α·∇J(θ)     [175 Milliarden Parameter]
```

## 🎯 Zusammenfassung - Was haben wir gelernt?

### ✅ **Kernkonzepte verstanden:**

1. **MSE-Landschaft**: Jede Parameter-Kombination hat einen Fehlerwert
2. **Gradientenabstieg**: Folge dem steilsten Abstieg zum Minimum  
3. **Iteration beats Perfection**: Schritt für Schritt zum Optimum
4. **Universelles Prinzip**: Von 2 Parametern zu Milliarden

### 🔧 **Praktische Erkenntnisse:**

- **Gradientenabstieg funktioniert**: Findet das gleiche Optimum wie analytische Lösung
- **Visualisierung hilft**: MSE-Landschaft macht Optimierung greifbar
- **Parameter vs. Hyperparameter**: m,b werden gelernt; α,Iterationen werden gewählt
- **Lernrate ist wichtig**: Zu klein→langsam, zu groß→instabil

### 🚀 **Ausblick auf nächste Vorlesungen:**

**Vorlesung 08 - Overfitting:**
- Mehr Parameter (Polynome)
- Overfitting-Problem  
- Train/Test-Split
- Praktische Anwendung mit echten Daten

### 💡 **Die wichtigste Erkenntnis:**

**"Das gleiche Prinzip funktioniert bei ChatGPT mit 175 Milliarden Parametern!"**

Der Gradient zeigt immer den Weg zum besseren Modell - egal ob 2 Parameter oder 2 Milliarden! 🧠✨